In [5]:
input_file = "input.txt"
from collections import namedtuple
import pandas as pd
import numpy as np

Equation = namedtuple('Equation', ['result', 'input'])

def parse_input(line):
    result, input = line.strip().split(": ")
    result = int(result)
    input = input.split(" ")
    input = list(map(int, input))
    return Equation(result=result, input=input)


with open(input_file) as file:
    lines = file.readlines()
    equations = list(map(parse_input, lines))

# Approach 1: Brute Force
Worked for Task 1

In [6]:
def operator_to_number_string(operator):
    if operator == "+":
        return "0"
    if operator == "*":
        return "1"
    if(operator == ""):
        return "2"

def number_string_to_operator(number_string):
    if number_string == "0":
        return "+"
    if number_string == "1":
        return "*"
    if(number_string == "2"):
        return ""

def get_next_operators(operators, base):
    max_number = pow(base, len(operators))-1
    binary_operators = list(map(operator_to_number_string, operators))
    binary_operators = ''.join(binary_operators)
    operator_number = int(binary_operators, base)
    next_operator_number = operator_number + 1
    if next_operator_number > max_number:
        return None
    next_operator_binary = np.base_repr(next_operator_number, base=base).zfill(len(operators))
    return list(map(number_string_to_operator, list(next_operator_binary)))

def calculate(input, operators, expected_result):
    result = input[0]
    for index, value in enumerate(operators):
        result = eval(str(result)+value+str(input[index+1]))
        if result > expected_result:
            return None
    return result


def get_starting_operators(input):
    return ["+"]*(len(input) - 1)


# sum_of_results = 0
# for equation in equations:
#     can_be_solved = False
#     operators = get_starting_operators(equation.input)
#     while not can_be_solved and operators != None:
#         # print(operators)
#         # print(result)
#         # print("____")
#         result = calculate(equation.input, operators, equation.result)
#         if result == equation.result:
#             can_be_solved = True
#             sum_of_results += result
#         else:
#             # 2 for part 1, 3 for part 2
#             operators=get_next_operators(operators, 2)

In [7]:
equations = pd.Series(equations)
equations.map(lambda x: len(x.input)).value_counts()

5     242
6     122
7      95
9      82
8      80
12     77
11     59
10     58
4      23
3      12
Name: count, dtype: int64

# Approach 2: Tree with cancellation



In [8]:
from typing import List
CachedEvaluation = namedtuple('CachedEvaluation', ['operator', 'last_result', 'operator_history'])

def evaluate_operators(equation: Equation, evaluation: CachedEvaluation):
    result = evaluation.last_result

    next_number_index = len(evaluation.operator_history) + 1

    next_number = equation.input[next_number_index]

    if evaluation.operator == "+":
        return result + next_number
    
    if evaluation.operator == "*":
        return result * next_number
    
    if evaluation.operator == "||":
        return int(str(result) + str(next_number))

def evalute_equation(equation: Equation, available_operators: List[str]):
    evaluations = list(map(lambda operator: CachedEvaluation(operator=operator, last_result=equation.input[0], operator_history=[]), available_operators)) 
    while evaluations:
        #depth-search
        evaluation = evaluations.pop()
        result = evaluate_operators(equation, evaluation)
        operators_of_result = evaluation.operator_history + [evaluation.operator]

        if result == equation.result and len(operators_of_result) == len(equation.input)-1:
            return True
        if result <= equation.result and len(operators_of_result) < len(equation.input)-1:
            for operator in available_operators:
                evaluations.append(CachedEvaluation(operator=operator, last_result=result, operator_history=operators_of_result))
    return False


calibration_result = 0
for index, equation in enumerate(equations):
    if evalute_equation(equation, ["+", "*", "||"]):
        calibration_result += equation.result
 
calibration_result



581941094529163